In [1]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *


# from pathlib import Path
import h5py

Using TensorFlow backend.


加载模型权重

In [2]:
model_weight_resnet50 = "model_ResNet50.h5"
model_weight_xception = "model_Xception.h5"
model_weight_inceptionV3 = "model_InceptionV3.h5"

建立辅助函数，提取特征向量

In [2]:
def write_gap(MODEL,model_weight, image_size, preprocessing=None, filename_option=""):
    # parameters
    batch_size = 64
    # path_dataset = Path('dataset')
    path_train = "dataset/train2"
    path_test = "dataset/valid2"
    
    # set up Model
    print('set up Model')
    height = image_size[0]
    width = image_size[1]
    
    base_model = MODEL(input_tensor=Input((height, width, 3)),
                            weights=None, 
                            include_top=False, 
                            input_shape=(height, width, 3)
                           )
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))
    
    # load weight
    print('load weight')
    model.load_weights("models/" + model_weight, by_name=True)
    
    train_gen = ImageDataGenerator(
        rotation_range=10.,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.1,
        zoom_range=0.1,
        preprocessing_function=preprocessing
    )
    
    test_gen = ImageDataGenerator(
        preprocessing_function=preprocessing
    )
    
    print('read dataset')
    train_generator = train_gen.flow_from_directory(
        path_train, (height, width), shuffle=False, batch_size=batch_size)
    test_generator = test_gen.flow_from_directory(
        path_test, (height, width), shuffle=False, batch_size=batch_size)
    
    # predictions
    print('make predictions')
    steps_train_sample = train_generator.samples // batch_size + 1
    steps_test_sample = test_generator.samples // batch_size + 1
    train = model.predict_generator(train_generator, steps=steps_train_sample)
    test = model.predict_generator(test_generator, steps=steps_test_sample)
    
    # save features
    
    print('save features')
    path_finetune_feature = 'features/finetune_{}{}.h5'.format(MODEL.__name__, filename_option)
    with h5py.File(path_finetune_feature) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("valid", data=test)
        h.create_dataset("label", data=train_generator.classes)
        h.create_dataset("valid_label", data=test_generator.classes)
    print("write_gap {} finsh".format(Model.__name__))

In [6]:
# ResNet50
model_weight_resnet50 = "model_ResNet50_BN_160.h5"
write_gap(ResNet50, model_weight_resnet50, (224, 224),  filename_option="BN_160_2")

set up Model
load weight
read dataset
Found 20864 images belonging to 10 classes.
Found 1560 images belonging to 10 classes.
make predictions
save features
write_gap Model finsh


In [3]:
# VGG16
model_weight_vgg16 = "model_VGG16_BN_15_SGD.h5"
write_gap(VGG16, model_weight_vgg16, (224, 224), filename_option="BN_15_SGD")

set up Model
load weight
read dataset
Found 20864 images belonging to 10 classes.
Found 1560 images belonging to 10 classes.
make predictions
save features
write_gap Model finsh


In [ ]:
model_filename = 'model_InceptionV3_BN_200_SGD.h5'
write_gap(InceptionV3, model_filename, (299, 299), preprocessing=inception_v3.preprocess_input, filename_option='BN_200_SGD')

In [8]:
def write_gap_test(MODEL, model_weight, image_size, preprocessing=None, filename_option=""):
    batch_size = 64
    path_test = "dataset/to_prediction"
    print('set up Model')
    height = image_size[0]
    width = image_size[1]
    base_model = MODEL(input_tensor=Input((height, width, 3)),
                            weights=None, 
                            include_top=False, 
                            input_shape=(height, width, 3)
                           )
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))
    # load model
    print('load weight')
    model.load_weights("models/" + model_weight, by_name=True)
    
    
    gen = ImageDataGenerator(preprocessing_function=preprocessing)
    
    test_generator = gen.flow_from_directory(
            path_test, (height, width), shuffle=False, batch_size=batch_size, class_mode=None)
    steps = test_generator.samples//batch_size+1
    test = model.predict_generator(test_generator, steps)
    path_finetune_feature = 'features/finetune_test_{}{}.h5'.format(MODEL.__name__, filename_option)
    with h5py.File(path_finetune_feature) as h:
        h.create_dataset("test", data=test)
    print("write_gap {} successed".format(Model.__name__))

In [8]:
# ResNet
model_weight_resnet50 = "model_ResNet50_BN_160.h5"
write_gap_test(ResNet50, model_weight_resnet50, (224, 224), filename_option = "_BN160")

set up Model
load weight
Found 79726 images belonging to 1 classes.
write_gap Model successed


In [9]:
# VGG16
model_weight_vgg16 = "model_VGG16_BN_15_20epoche.h5"
write_gap_test(VGG16, model_weight_vgg16, (224, 224), filename_option = "_BN15")

set up Model
load weight
Found 79726 images belonging to 1 classes.
write_gap Model successed


In [9]:
# VGG16
model_weight_vgg16 = "model_VGG16_BN_15_SGD.h5"
write_gap_test(VGG16, model_weight_vgg16, (224, 224), filename_option = "_BN15_SGD")

set up Model
load weight
Found 79726 images belonging to 1 classes.
write_gap Model successed


In [13]:
model_weight_inceptionV3 = "model_InceptionV3_BN_200.h5"
write_gap_test(InceptionV3, model_weight_inceptionV3, (299, 299), preprocessing=inception_v3.preprocess_input, filename_option="BN_200")

set up Model
load weight
Found 79726 images belonging to 1 classes.
write_gap Model successed
